In [ ]:
import pandas as pd
import sqlite3
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import os
import numpy as np

print("Đã import các thư viện ")

In [ ]:
BASE_DIR = r"BookInsight\data"

SQL_DB_PATH = r"D:\BookInsight\data\database\books.db"

VS_DIR = r"D:\BookInsight\data\vectorstores"
FAISS_INDEX_PATH = os.path.join(VS_DIR, "index.faiss")
EMBEDDINGS_PATH = os.path.join(VS_DIR, "embeddings.npy")
META_PATH = os.path.join(VS_DIR, "unique_ids.pkl")

print("Cấu hình đường dẫn xong")
print(f"File SQL: {SQL_DB_PATH}")
print(f"Thư mục Vector store: {VS_DIR}")

In [ ]:
# Tải model embedding
try:
    model_name = "BAAI/bge-small-en-v1.5"
    print("Đang tải model...")

    model = SentenceTransformer(model_name, device='cpu')
    print("Tải model xong")
except Exception as e:
    print("Lỗi khi tải model")


In [ ]:
# Load Nền tảng Dữ liệu
try:
    print(f"Đang load FAISS index từ: {FAISS_INDEX_PATH}...")
    index = faiss.read_index(FAISS_INDEX_PATH)
    print(f"-> Index loaded. Tổng số vectors: {index.ntotal}")

    print(f"Đang load metadata (unique_ids) từ: {META_PATH}...")
    with open(META_PATH, 'rb') as f:

        unique_ids_list = pickle.load(f)
    print(f"-> Metadata loaded. Tổng số IDs: {len(unique_ids_list)}")

    print(f"Đang kết nối tới SQL DB tại: {SQL_DB_PATH}...")
    conn = sqlite3.connect(SQL_DB_PATH)

    conn.row_factory = sqlite3.Row
    print("Kết nối SQL thành công.")

    print("\nNỀN TẢNG (BASE RETRIEVER) ĐÃ SẴN SÀNG")

except Exception as e:
    print(f"Lỗi khi load nền tảng: {e}")

# BASE RETRIEVER

In [ ]:

def search_vector_store(query_text, k=5):
    """
    1. Embed câu hỏi.
    2. Dùng FAISS để tìm k vị trí (indices) gần nhất.
    3. Dùng metadata (unique_ids_list) để lấy ra các unique_id thực tế.
    """
    print(f"Đang embed câu hỏi: '{query_text}'")
    query_with_instruction = f"query: {query_text}"

    query_vector = model.encode([query_with_instruction], normalize_embeddings=True)
    query_vector = np.array(query_vector).astype('float32')

    print(f"Đang tìm kiếm {k} kết quả gần nhất trong FAISS...")
    D, I = index.search(query_vector, k)

    retrieved_indices = I[0]

    retrieved_unique_ids = [unique_ids_list[i] for i in retrieved_indices]

    print(f"-> Đã tìm thấy IDs: {retrieved_unique_ids}")
    return retrieved_unique_ids, retrieved_indices, D[0]

def get_book_details_from_sql(unique_ids):
    """
    Truy vấn SQL DB để lấy thông tin chi tiết của sách từ list unique_id.
    """
    print(f"Đang truy vấn SQL cho {len(unique_ids)} IDs...")

    placeholders = ', '.join('?' for _ in unique_ids)
    query_sql = f"SELECT * FROM books WHERE unique_id IN ({placeholders})"

    cursor = conn.cursor()
    cursor.execute(query_sql, unique_ids)

    results = [dict(row) for row in cursor.fetchall()]
    results_sorted = sorted(results, key=lambda x: unique_ids.index(x['unique_id']))

    print(f"-> Đã lấy được {len(results_sorted)} chi tiết sách.")
    return results_sorted

print("Đã định nghĩa hàm search_vector_store() và get_book_details_from_sql().")

In [ ]:
test_query = "AI books for beginners"
TOP_K = 5

try:

    retrieved_ids, positions, distances = search_vector_store(test_query, k=TOP_K)

    book_details = get_book_details_from_sql(retrieved_ids)

    print("\n" + "="*30)
    print(f"KẾT QUẢ TÌM KIẾM CHO: '{test_query}'")
    print("="*30)

    for i, book in enumerate(book_details):
        print(f"\n--- Kết quả #{i+1} (Vị trí: {positions[i]}, Khoảng cách: {distances[i]:.4f}) ---")
        print(f"  ID:    {book['unique_id']}")
        print(f"  Title: {book['title']}")
        print(f"  Author: {book['author_name']}")
        print(f"  Rating: {book['average_rating']} ({book['rating_number']} reviews)")

except Exception as e:
    print(f"Lỗi khi chạy thử: {e}")

# Nhận xét kết quả của base rag:
- Kết quả 1: Tốt
Introduction to Artificial Intelligence
- Kết quả 2: Không tốt
The Ultimate Fantasy AI Art Book nói về AI Art không liên quan nói về Art không phải là Sách về AI cho người mới bắt đầu

### Vấn đề: model đã thấy "AI" và "Book" cho rằng có liên quan tuy nhiên bỏ lỡ context (for beginners) bị nhiễu bỏi từ "Art"


# RAG-Fusion

In [ ]:
! pip install openai

In [ ]:
import openai

print("Đã cài đặt và cấu hình OpenAI")

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = openai.api_key)

def expand_query_with_llm(query, num_queries=4):
    """
    Dùng LLM (GPT) để tạo ra các biến thể của câu hỏi.
    Đây là bước đầu tiên của RAG-Fusion.
    """
    print(f"Đang mở rộng câu hỏi: '{query}'...")

    prompt = f"""
    You are a helpful assistant. Your task is to generate {num_queries} different search queries
    that are semantically similar to the original query.
    The queries should be diverse and cover different aspects or phrasings of the original query.

    Original Query: {query}

    Please provide ONLY a Python list of the new queries, like this:
    ["query 1", "query 2", "query 3", "query 4"]
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )

        generated_queries_str = response.choices[0].message.content

        generated_queries = eval(generated_queries_str)

        all_queries = [query] + generated_queries

        print(f"-> Các câu hỏi đã mở rộng: {all_queries}")
        return all_queries

    except Exception as e:
        print(f"Lỗi khi gọi OpenAI: {e}")
        return [query]

print("Đã định nghĩa hàm expand_query_with_llm().")

In [ ]:

test_query = "AI books for beginners"

try:
    expanded_queries = expand_query_with_llm(test_query, num_queries=4)

    print("\n--- KẾT QUẢ MỞ RỘNG ---")
    for q in expanded_queries:
        print(q)

except Exception as e:
    print(f" Lỗi khi chạy thử expander: {e}")

# RRF (reciprocal Rank Fusion)


In [ ]:
def reciprocal_rank_fusion(search_results_list, k=60):
    """
    Thực hiện RRF trên một danh sách các kết quả tìm kiếm.
    search_results_list: List của các list vị trí (indices),
                         ví dụ: [ [pos1, pos2], [pos1, pos3], ... ]
    k: Hằng số RRF (thường là 60, như trong repo)
    """
    # Dùng dictionary để lưu trữ điểm RRF của từng document (vị trí)
    scores = {}

    # Lặp qua từng danh sách kết quả (từ mỗi câu hỏi)
    for results in search_results_list:
        # Lặp qua từng document (vị trí) trong danh sách kết quả đó
        for rank, pos in enumerate(results):
            # Tính điểm RRF cho document này
            # rank + 1 vì rank bắt đầu từ 0
            doc_score = 1.0 / (k + (rank + 1))

            # Nếu document chưa có trong 'scores', thêm vào
            if pos not in scores:
                scores[pos] = 0.0

            # Cộng điểm RRF vào document
            scores[pos] += doc_score

    # Sắp xếp các document theo điểm RRF giảm dần
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

    # Trả về list các (vị trí, điểm_rrf) đã sắp xếp
    return sorted_scores

print("Đã định nghĩa hàm reciprocal_rank_fusion().")

In [ ]:
print("Bắt đầu chạy thử RAG-Fusion...")

test_query = "AI books for beginners"
TOP_K = 5

try:
    all_queries = expand_query_with_llm(test_query, num_queries=4)

    all_search_results = []

    print("\nĐang thực hiện tìm kiếm song song cho 5 câu hỏi...")
    for q in all_queries:
        _, positions, _ = search_vector_store(q, k=TOP_K)
        all_search_results.append(positions)
        print(f"-> Hoàn thành search cho: '{q}'")

    print("\nĐang hợp nhất kết quả với RRF...")
    fused_results = reciprocal_rank_fusion(all_search_results)

    final_positions = [pos for pos, score in fused_results[:TOP_K]]
    final_scores = [score for pos, score in fused_results[:TOP_K]]

    final_unique_ids = [unique_ids_list[i] for i in final_positions]

    book_details = get_book_details_from_sql(final_unique_ids)

    print("\n" + "="*30)
    print(f"KẾT QUẢ RAG-FUSION CHO: '{test_query}'")
    print("="*30)

    for i, book in enumerate(book_details):
        print(f"\n--- Kết quả RAG-FUSION #{i+1} (Vị trí: {final_positions[i]}, Điểm RRF: {final_scores[i]:.4f}) ---")
        print(f"  ID:    {book['unique_id']}")
        print(f"  Title: {book['title']}")
        print(f"  Author: {book['author_name']}")
        print(f"  Rating: {book['average_rating']} ({book['rating_number']} reviews)")

except Exception as e:

    print(f"Lỗi khi chạy RAG-Fusion: {type(e)} - {str(e)[:500]}")

# Test pipeline

In [1]:
import os


In [1]:
import sys
import os
import json


PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

SRC_PATH = os.path.join(PROJECT_ROOT, 'src')

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Đã thêm '{SRC_PATH}' vào sys.path")
else:
    print(f"'{SRC_PATH}' đã có trong sys.path")


try:
    from retriever.smart_retriever import SmartRetriever
    print("Import SmartRetriever thành công.")
except ImportError as e:
    print(f"LỖI IMPORT: {e}")
    print("Nếu bạn thấy lỗi này, hãy Restart Kernel và chạy lại.")
except Exception as e:
    print(f"Lỗi không mong muốn: {e}")

print("\n--- Đang khởi tạo SmartRetriever ---")
try:
    retriever = SmartRetriever(device='cpu')
    print("\nSmartRetriever đã sẵn sàng")
except NameError:
    print("LỖI: Class 'SmartRetriever' không thể import. Vui lòng kiểm tra lỗi import ở trên.")
except Exception as e:
    print(f"Lỗi khi khởi tạo SmartRetriever: {type(e)} - {e}")

Đã thêm 'd:\BookInsight\src' vào sys.path
t
Import SmartRetriever thành công.

--- Đang khởi tạo SmartRetriever ---
[SmartRetriever] Đang khởi tạo các thành phần...
[VectorStore] Đang khởi tạo...
[VectorStore] Đang tải Model Embedding: BAAI/bge-small-en-v1.5
[VectorStore] Đang tải FAISS index từ: d:\BookInsight\data\vectorstores\index.faiss
[VectorStore] Đang tải metadata từ: d:\BookInsight\data\vectorstores\unique_ids.pkl
 [VectorStore] Khởi tạo hoàn tất. Sẵn sàng tìm kiếm.
[SQLDatabase] Đang kết nối tới: d:\BookInsight\data\database\books.db
[SQLDatabase] Kết nối thành công.
[QueryExpander] Đã khởi tạo với model: gpt-3.5-turbo
[SmartRetriever] Khởi tạo hoàn tất.

SmartRetriever đã sẵn sàng


In [2]:
test_query = "AI books for beginners"
TOP_K = 5

print(f"--- Đang chạy truy vấn: '{test_query}' ---")
try:
    results = retriever.retrieve(test_query, top_k=TOP_K)

    print("\n--- KẾT QUẢ CUỐI CÙNG ---")
    print(json.dumps(results, indent=2))

except Exception as e:
    print(f"Lỗi khi đang truy vấn: {type(e)} - {e}")

# (Tùy chọn) Đóng kết nối SQL khi bạn test xong
# retriever.close()

--- Đang chạy truy vấn: 'AI books for beginners' ---

[SmartRetriever] Bắt đầu truy vấn cho: 'AI books for beginners'
[QueryExpander] Đang mở rộng câu hỏi: 'AI books for beginners'...
-> [QueryExpander] Các câu hỏi đã mở rộng: ['AI books for beginners', 'Best artificial intelligence books for beginners', 'Introductory artificial intelligence books', 'Top AI books for novices', 'Recommended AI reading materials for beginners']
[SmartRetriever] Đang thực hiện tìm kiếm song song...
[SmartRetriever] Đang hợp nhất kết quả với RRF...
[SmartRetriever] Truy vấn RAG-Fusion hoàn tất.

--- KẾT QUẢ CUỐI CÙNG ---
[
  {
    "unique_id": "048624864X",
    "title": "Introduction to Artificial Intelligence: Second, Enlarged Edition (Dover Books on Mathematics)",
    "author_name": "Philip C. Jackson",
    "publisher": "Dover; Second Edition, Enlarged (June 1, 1985)",
    "publication_year": -1.0,
    "main_category": "Books",
    "categories": "Books > Computers & Technology > Computer Science",
    "b

In [4]:
test_query = "AI books for beginners"
TOP_K = 8

print(f"--- Đang chạy truy vấn: '{test_query}' ---")
try:
    results = retriever.retrieve(test_query, top_k=TOP_K)
    
   
    print("\n--- KẾT QUẢ CUỐI CÙNG (TÓM TẮT) ---")
    print(f"Tìm thấy {len(results)} kết quả:")
    
    for i, book in enumerate(results):
        print(f"  #{i+1}: {book['title']} (Score: {book['rrf_score']:.4f})")
    
except Exception as e:
    print(f"Lỗi khi đang truy vấn: {type(e)} - {e}")

# (Tùy chọn) Đóng kết nối SQL khi bạn test xong
# retriever.close()

--- Đang chạy truy vấn: 'AI books for beginners' ---

[SmartRetriever] Bắt đầu truy vấn cho: 'AI books for beginners'
[QueryExpander] Đang mở rộng câu hỏi: 'AI books for beginners'...
-> [QueryExpander] Các câu hỏi đã mở rộng: ['AI books for beginners', 'Best artificial intelligence books for beginners', 'Introductory artificial intelligence book recommendations', 'Top AI books suitable for beginners', 'Beginner-friendly artificial intelligence reading list']
[SmartRetriever] Đang thực hiện tìm kiếm song song...
[SmartRetriever] Đang hợp nhất kết quả với RRF...
[SmartRetriever] Truy vấn RAG-Fusion hoàn tất.

--- KẾT QUẢ CUỐI CÙNG (TÓM TẮT) ---
Tìm thấy 8 kết quả:
  #1: Introduction to Artificial Intelligence: Second, Enlarged Edition (Dover Books on Mathematics) (Score: 0.0820)
  #2: Introduction to Deep Learning (The MIT Press) (Score: 0.0799)
  #3: AIQ: How People and Machines Are Smarter Together (Score: 0.0760)
  #4: Machine Learning Refined: Foundations, Algorithms, and Applicatio

### Kết quả cải thiện hoàn toàn so với base-retriever
kết quả có liên quan đến AI